In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import classification_report,precision_score,confusion_matrix

In [2]:
dataset = datasets.load_breast_cancer()
X,y = dataset.data[0:100],dataset.target[0:100]
sigma = 2
gamma = 1
x = preprocessing.StandardScaler().fit_transform(X)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=1)

In [5]:
def featureSelction(x,y):
    clf = LinearSVC(penalty='l1',C=1.0,dual=False)
    clf.fit(x,y)
    
    model = SelectFromModel(clf,prefit=True)
    tmp = model.transform(x)
    common = []
    for index in range(x.shape[1]):
        if x[0,index] in list(tmp[0]):
            common.append(index)
    return common

In [6]:
common = featureSelction(x_train,y_train)
x_train = x_train[:,common]
x_test = x_test[:,common]

In [7]:
def FOA():
    x_asis = np.random.random((1,2))
    y_asis = np.random.random((1,2))
    samples = 20
    iter_times = 10
    def searchPosition(x_asis,y_asis):
        position = []
        smell = []
        for num in range(samples):
            x = x_asis+2*np.random.rand()-1
            y = y_asis+2*np.random.rand()-1
            params = 1/np.sqrt(x**2+y**2)
            accuracy = LSSVMClassifier(params[0,0],params[0,1])
            position.append((x,y))
        smell.append(accuracy)
        index = np.argmax(smell)
        bestX,bestY = position[index]
        bestSmall = smell[index]
        return bestX,bestY,bestSmell          
         
    start = time.time()
    print('---FOA初始化开始---')       
    bestX,bestY,bestSmell = searchPosition(x_asis,y_asis)
    print('time:',time.time()-start)
    def searchBestParams(bestX,bestY,bestSmell):
        smell = [bestSmell]
        for num in range(iter_times):
            print(num,':',bestSmell)
            x,y,accuracy = searchPosition(bestX,bestY)
            
        
        
            
            
            if accuracy > bestSmell:
                bestX,bestY,bestSmell= x,y,accuracy
            smell.append(bestSmell)
        params = 1/np.sqrt((bestX**2+bestY**2))
        print(params[0])
        print('accuracy by LSSVM:',bestSmell)
    begin = time.time()
    print('---FOA寻优开始---')
    searchBestParams(bestX,bestY,bestSmell)
    print('end:',time.time()-begin)

SyntaxError: invalid character in identifier (<ipython-input-7-c6aba35380b6>, line 2)

In [ ]:
def LSSVMClassifier(sigma,gamma):
    def kernel(x,y,sigma):
        return np.exp(-np.sum((x-y)**2,axis=0)/2*sigma**2)
    def matrixOmega(x,y,sigma):
        num = len(x)
        mat = np.zeros((num,num))
        for i in range(num):
            for j in range(num):
                k = y[i]*y[j]*kernel(x[i],x[j],sigma)
                mat[i,j] = k
        return mat   
    mat = matrixOmega(x_train,y_train,sigma)
    def getparams(x_train,y_train,mat,gamma,sigma):
        length = len(x_train)+1
        A = np.zeros((length,length))
        A[0,1:length] = y_train.T
        A[1:length,0] = y_train
        A[1:length,1:length] = mat+np.eye(length-1)/gamma
        B = np.zeros((length,1))
        B[1:length] = np.ones((length-1,1))
        result = np.linalg.solve(A,B)
        a,b = result[0:-1],result[-1]
        return a,b
    a,b = getparams(x_train,y_train,mat,gamma,sigma)
    def predictTestData(x_test,y_test,a,b):
        result = np.zeros(len(x_test))
        for i in range(len(x_test)):
            result[i] = b[0]
            for j in range(len(x_train)):
                result[i] += a[i]*y_train[i]*kernel(x_test[i],x_train[j],sigma)
                result[i] = np.sign(result[i])
                accuracy = precision_score(y_test,result)
        return accuracy      
    accuracy = predictTestData(x_test,y_test,a,b)
    return accuracy

In [ ]:
FOA()

In [ ]:
def trainModelBySVC():
    clf = SVC()
    clf.fit(x_train,y_train)
    predict = clf.predict(x_test)
    print('accuracy by SVM:',precision_score(y_test,predict))

In [ ]:
trainModelBySVC()